# Lab 02

Представлен набор данных о городских квартирах. Рассчитайте следующие величины:

* Оценку отношения R параметров 'Полная площадь квартиры' (full_sq) и 'Количество комнат' (num_room);
* стандартную ошибку данной оценки;
* долю p квартир, у которых 'Материал стен' = 4;
* левую и правые доверительные границы нормальной аппроксимации p, доверительная вероятность = 95.3%.

Объем генеральной совокупности равен 41468.    

*Примечание*. Под дисперсией и стандартным отклонением подразумеваются их выборочные (несмещенные) аналоги.
    
Итоговый файл **.csv** должен содержать 5 чисел, через запятую.

In [1]:
from math import sqrt
from pathlib import Path
from typing import List, Any

import numpy as np

from scipy.stats import norm
import pandas as pd

In [2]:
N = 41468
alpha = 1- (95.3/100)

file_path = Path('.')
data_path = file_path / '..' / 'data' / 'lw2' / 'projects' / '1032173841' / 'data.csv'
data_path = data_path.resolve()

data = pd.read_csv(data_path)
n = len(data)
f = n/N

display(data)

,full_sq,life_sq,material,num_room
0,41,27,2,2
1,53,30,1,2
2,46,29,1,2
3,46,30,1,2
4,38,19,2,1
...,...,...,...,...
7406,39,17,1,1
7407,81,41,4,2
7408,32,18,1,1
7409,40,25,1,2


In [3]:
def mean(lst: List[float]) -> float:
    return sum(lst)/len(lst)

R = mean(data.full_sq) / mean(data.num_room)
print("R", R)

R 27.356736111111115


In [4]:
def std_err(X, Y, R):
    err = (
            sqrt(1-f)     * sqrt(sum([(y_i - R*x_i)**2 for x_i, y_i in zip(X, Y)])) /
#       ---------------------------------------------------------------------------
        (sqrt(n) * mean(X) * sqrt(n-1))
    )
    return err
std = std_err(data.num_room, data.full_sq, R)
print("std_err", std)

std_err 0.08263028105835243


In [5]:
def median(lst: List[float]) -> float:
    sort = sorted(lst)
    mid = (len(lst) - 1) // 2
    
    if (len(lst) % 2) == 0:
        return float(sort[mid])
    else:
        return float((sort[mid] + sort[mid + 1])/2)

In [6]:
def var(lst: List[float]) -> float:
    var = (
        sum([(l_i - mean(lst))**2 for l_i in lst]) / 
    #   ------------------------------------------
                         (len(lst)-1)
    )
    return var

In [7]:
a = len([m for m in data.material if m == 4])
p = a/n
q = 1 - p
print("a", a)
print("p", p)

a 438
p 0.059101335852111726


In [8]:
t_l = norm.ppf(alpha / 2)
t_r = norm.ppf((1-alpha) / 2)

di_l = p - (t_l*sqrt(1-f)*sqrt(p*q/(n-1)) + 1/(2*n))
di_r = p + (t_r*sqrt(1-f)*sqrt(p*q/(n-1)) + 1/(2*n))

print(di_l, di_r)

0.06396506733080286 0.05902247871145574


In [9]:
data = {
    'R':     [R],
    's_R':   [std],
    'p':     [p],
    'left':  [di_l],
    'right': [di_r],
}

df = pd.DataFrame(data=data)
display(df)

csv_path = Path('.') / 'lab02.csv'
csv_path = csv_path.resolve()

if csv_path.exists():
    csv_path.unlink()

df.to_csv(csv_path, index=False)

,R,s_R,p,left,right
0,27.356736,0.08263,0.059101,0.063965,0.059022
